# Investing On ETFs Using The Kelly Formula Part 2

## How To Allocate Capital Using The Kelly Formula

## Introduction
The investor's main problem is identifying profitable trading strategies.
He could deploy fundamental, technical and quantitative techniques to pick winning trading systems.
The investor's next problem is how to allocate capital to these different profit-generating trading strategies.

In this article, I will present the Kelly formula which is a popular allocation system used in both sports betting and investing. I will use the Vanguard ETFs data sourced in the previous article to calculate the Kelly leverages (or fractions) for each ETF on each group. I will analyze the results for the different cases considered: unconstrained, full and half Kelly leverages.

## The Kelly Formula
The Kelly formula was presented by Bell labs researcher John Kelly in his paper "A New Interpretation Of Information Rate". He introduced a betting system maximizing the expected logarithm of wealth, which is equivalent to maximizing expected compounded growth rate. John Kelly illustrated the formula's application in horse racing where bet size per horse is adjusted to the bettor's informed estimate of the horse's chance of winning.

The Kelly formula was introduced to the public by mathematician, blackjack researcher and quantitative hedge fund pioneer Edward Thorp. In 1960s, he combined a card-counting system and the Kelly formula to beat dealers in the blackjack tables of Las Vegas. Later, Edward Thorp used statistical analysis and the Kelly formula to beat other investment firms in Wall Street.

## The Kelly Formula In Investing
In this section, we follow Thorp's [paper](https://www.researchgate.net/publication/247922818_The_Kelly_Criterion_in_Blackjack_Sports_Betting_and_the_Stock_Market) showcasing the Kelly formula's application in stock market investing.   

### Case 1: Single Security
The objective is to maximize the growth rate $g$ of a portfolio composed of a single security
$$ \max_{f} g(f), $$

where growth rate $g$ is
$$ g(f) = r_{Risk-free} + f m - \frac{1}{2} f^2 s^2, $$

and $f$ is the fraction of capital we allocate to the security.

Assuming that the security follows a Normal distribution with mean $m$ and variance $s^2$, the optimal allocation $f^*$ is 
$$ f^* = \frac{m}{s^2}, $$

and the optimum growth rate is 
$$ g(f^*) = r_{Risk-free} + \frac{1}{2} f^{*2} s^2. $$

### Case 2: Multiple Securities
We extend the previous case to maximize the growth rate $g$ of a portfolio composed of multiple securities
$$ \max_{F} g(F), $$

where growth rate $g$ is
$$ g(F) = r_{Risk-free} + F^T M - \frac{1}{2} F^T C F, $$

and $F$ is a vector containing fractions of capital we allocate to each security
$$ F = \left( f_1, f_2, ..., f_n \right). $$

Assuming that each security $i$ follows a Normal distribution, $M$ is the vector containing average of excess returns of each security
$$ M = \left( m_1, m_2,..., m_n \right), $$

and $C$ is the covariance matrix of the returns between securities $i$ and $j$
$$
\begin{pmatrix} 
c_{11} & c_{12} & ... & c_{1n} \\ 
c_{21} & c_{22} & ... & c_{2n} \\ 
\vdots & \vdots & \ddots & \vdots \\ 
c_{n1} & c_{n2} & ... & c_{nn} \\ 
\end{pmatrix}
$$

the optimal allocation $F^*$ is
$$ F^* = C^{-1} M, $$

and the optimum growth rate is
$$ g(F^*) = r_{Risk-free} + \frac{1}{2} F^T C F. $$

## Step By Step
We apply the results above to allocate capital to different Vanguard ETF groups like US bonds, US stocks, international bodns, international stocks and sectors.
1. Calculate the Kelly leverage per ETF per group.
2. Analysis of the results.

## Calculate The Kelly Leverage Per ETF Per Group

Import packages

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

Set group keys

In [2]:
groups = ['us bonds',
          'us stocks',
          'intl bonds',
          'intl stocks',
          'sectors']

Set input input files

In [3]:
input = {'us bonds': 'etf_us_bonds.pickle', 
         'us stocks': 'etf_us_stocks.pickle',
         'intl bonds': 'etf_intl_bonds.pickle',
         'intl stocks': 'etf_intl_stocks.pickle',
         'sectors': 'etf_sectors.pickle'}

Create output dictionary

In [4]:
output = {'us bonds': {},
          'us stocks': {},
          'intl bonds': {},
          'intl stocks': {},
          'sectors': {}}

Set market parameters

In [5]:
risk_free = 0.025
max_leverage = 4.00

Calculate Kelly leverage of each ETF group

In [6]:
for i in groups:    

    # Load file
    with open(input[i], 'rb') as f:
        close = pickle.load(f)
    f.close()
    
    # Daily returns for the past 6 months
    returns = close[-120:].pct_change()
    
    # Excess daily returns
    excess_returns = returns - risk_free / 250
    
    # Mean excess daily returns annualized
    M = excess_returns.mean() * 250
    
    # Covariance of daily returns
    C = returns.cov() * 250
    
    # Kelly leverage
    F = np.matmul(np.linalg.inv(C), M)
    
    # Adjust leverage
    adj_leverage = max_leverage / np.sum(np.abs(F))
    
    # Leverages for unconstrained, full and half Kelly
    Kelly = [F, 
             adj_leverage * F, 
             0.5 * adj_leverage * F] 
    
    Kelly_id = ['unconstrained',
                'full',
                'half']
    
    for k in range(len(Kelly)):
        # Growth rate 
        g = risk_free + np.matmul(np.transpose(Kelly[k]), M) - 0.5 * np.matmul(np.matmul(np.transpose(Kelly[k]), C), Kelly[k])
        # Sharpe ratio
        sharpe = np.sqrt(np.matmul(np.matmul(np.transpose(Kelly[k]), C), Kelly[k]))
        
        # Updated outputs: F, g and sharpe
        output[i][Kelly_id[k]] = Kelly[k]
        output[i][Kelly_id[k] + ' growth'] = g
        output[i][Kelly_id[k] + ' sharpe'] = sharpe
        
    # Update output
    output[i]['tickers'] = list(close.columns)

## Analysis Of The Results

### Vanguard US Bonds ETFs

The Kelly leverage for US Bonds ETFs are

In [7]:
group = 'us bonds'

pd.DataFrame({'Unconstrained': output[group]['unconstrained'],
              'Full Kelly': output[group]['full'],
              'Half Kelly': output[group]['half']},
             index=output[group]['tickers'])

,Unconstrained,Full Kelly,Half Kelly
EDV,12.156232,0.033435,0.016717
BIV,116.324525,0.319941,0.159971
VGIT,-94.226520,-0.259162,-0.129581
BLV,46.831168,0.128805,0.064403
VGLT,-21.563687,-0.059309,-0.029655
VMBS,-63.459954,-0.174542,-0.087271
BSV,-204.894055,-0.563545,-0.281772
VTIP,-90.945714,-0.250139,-0.125069
VGSH,-85.628605,-0.235515,-0.117757
BND,-46.758134,-0.128604,-0.064302


The growth rate and Sharpe ratio are

In [8]:
pd.DataFrame({'Unconstrained': [output[group]['unconstrained growth'], output[group]['unconstrained sharpe']],
              'Full Kelly': [output[group]['full growth'], output[group]['full sharpe']],
              'Half Kelly': [output[group]['half growth'], output[group]['half sharpe']]},
             index=['Growth Rate', 'Sharpe Ratio'])

,Unconstrained,Full Kelly,Half Kelly
Growth Rate,17.121054,0.118913,0.071989
Sharpe Ratio,5.847402,0.016083,0.008041


### Vanguard US Stocks ETFs

The Kelly leverage for US Stocks ETFs are

In [9]:
group = 'us stocks'

pd.DataFrame({'Unconstrained': output[group]['unconstrained'],
              'Full Kelly': output[group]['full'],
              'Half Kelly': output[group]['half']},
             index=output[group]['tickers'])

,Unconstrained,Full Kelly,Half Kelly
VIG,11.583435,0.041141,0.020571
VUG,282.321606,1.002729,0.501365
VYM,-20.176372,-0.071661,-0.035830
VV,-89.070723,-0.316355,-0.158177
MGC,45.122143,0.160262,0.080131
MGK,-215.370435,-0.764937,-0.382468
MGV,39.437699,0.140072,0.070036
VOO,-147.455077,-0.523720,-0.261860
VTI,104.591684,0.371481,0.185741
VTV,-22.634360,-0.080391,-0.040196


The growth rate and Sharpe ratio are

In [10]:
pd.DataFrame({'Unconstrained': [output[group]['unconstrained growth'], output[group]['unconstrained sharpe']],
              'Full Kelly': [output[group]['full growth'], output[group]['full sharpe']],
              'Half Kelly': [output[group]['half growth'], output[group]['half sharpe']]},
             index=['Growth Rate', 'Sharpe Ratio'])

,Unconstrained,Full Kelly,Half Kelly
Growth Rate,7.125607,0.075349,0.050197
Sharpe Ratio,3.768450,0.013385,0.006692


### Vanguard International Bonds ETFs

The Kelly leverage for International Bonds ETFs are

In [11]:
group = 'intl bonds'

pd.DataFrame({'Unconstrained': output[group]['unconstrained'],
              'Full Kelly': output[group]['full'],
              'Half Kelly': output[group]['half']},
             index=output[group]['tickers'])

,Unconstrained,Full Kelly,Half Kelly
BNDX,210.493973,2.705469,1.352734
VWOB,100.718599,1.294531,0.647266


The growth rate and Sharpe ratio are

In [12]:
pd.DataFrame({'Unconstrained': [output[group]['unconstrained growth'], output[group]['unconstrained sharpe']],
              'Full Kelly': [output[group]['full growth'], output[group]['full sharpe']],
              'Half Kelly': [output[group]['half growth'], output[group]['half sharpe']]},
             index=['Growth Rate', 'Sharpe Ratio'])

,Unconstrained,Full Kelly,Half Kelly
Growth Rate,16.462221,0.444818,0.235588
Sharpe Ratio,5.733624,0.073694,0.036847


### Vanguard International Stocks ETFs

The Kelly leverage for International Stocks ETFs are

In [13]:
group = 'intl stocks'

pd.DataFrame({'Unconstrained': output[group]['unconstrained'],
              'Full Kelly': output[group]['full'],
              'Half Kelly': output[group]['half']},
             index=output[group]['tickers'])

,Unconstrained,Full Kelly,Half Kelly
VT,11.873599,0.173985,0.086992
VEU,56.446795,0.827119,0.413560
VSS,-6.710062,-0.098323,-0.049162
VEA,47.170226,0.691189,0.345595
VGK,-10.031345,-0.146990,-0.073495
VPL,-52.156336,-0.764251,-0.382125
VNQI,26.673947,0.390855,0.195428
VXUS,-59.953430,-0.878502,-0.439251
VWO,1.964445,0.028785,0.014393


The growth rate and Sharpe ratio are

In [14]:
pd.DataFrame({'Unconstrained': [output[group]['unconstrained growth'], output[group]['unconstrained sharpe']],
              'Full Kelly': [output[group]['full growth'], output[group]['full sharpe']],
              'Half Kelly': [output[group]['half growth'], output[group]['half sharpe']]},
             index=['Growth Rate', 'Sharpe Ratio'])

,Unconstrained,Full Kelly,Half Kelly
Growth Rate,3.502137,0.126155,0.075764
Sharpe Ratio,2.637096,0.038642,0.019321


### Vanguard Sectors ETFs

The Kelly leverage for sectors ETFs are

In [15]:
group = 'sectors'

pd.DataFrame({'Unconstrained': output[group]['unconstrained'],
              'Full Kelly': output[group]['full'],
              'Half Kelly': output[group]['half']},
             index=output[group]['tickers'])

,Unconstrained,Full Kelly,Half Kelly
VOX,1.219117,0.050629,0.025314
VCR,-8.332042,-0.346022,-0.173011
VDC,-2.505247,-0.104041,-0.052020
VDE,-11.760600,-0.488407,-0.244203
VFH,17.209040,0.714675,0.357338
VHT,-19.652473,-0.816149,-0.408074
VIS,3.842607,0.159580,0.079790
VGT,14.855284,0.616926,0.308463
VAW,0.396723,0.016476,0.008238
VNQ,12.386444,0.514397,0.257199


The growth rate and Sharpe ratio are

In [16]:
pd.DataFrame({'Unconstrained': [output[group]['unconstrained growth'], output[group]['unconstrained sharpe']],
              'Full Kelly': [output[group]['full growth'], output[group]['full sharpe']],
              'Half Kelly': [output[group]['half growth'], output[group]['half sharpe']]},
             index=['Growth Rate', 'Sharpe Ratio'])

,Unconstrained,Full Kelly,Half Kelly
Growth Rate,6.271559,0.533054,0.281720
Sharpe Ratio,3.534560,0.146787,0.073393


## Conclusion
...